# USD vs. GBP

### Part I : Peeling out the monetary shock

In [1]:
Docker = "To peel out a monetary shocks TS, the data needed are:\n \
          * The federal funds rate series \n \
          * The inflation sequence \n \
          * The output gap sequence."
"""To get the output gap sequence, we use a quadratic function to fit the trend and determine the gap."""
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
from matplotlib import pyplot as plt
%matplotlib inline
import statsmodels.api as sm

def flashBack(old,n,mode="month"):
    #Get the back the argued period of time
    if mode == "month":
        #Flash back by month
        year = old.year - int((n - n%12)/12)
        res = n%12
        if res >= old.month:
            #We have to go back an extra year
            year = year - 1
            month = 12 + old.month - res
        else:
            month = old.month - res
        result = datetime(year,month,old.day)
    return result

def grabForward(target,domain):
    #Get the closest datetime to the target dt, in the domain time series indexed by the datetimess.
    tmp = domain[target:]
    try:
        return tmp.iloc[0,:].values
    except:
        return tmp.iloc[0]

def grabNearest(target,domain,output=False):
    #Get the colest datetime to the target dt.
    #Domain has to a datetime indexed Series or DataFrame.
    left = domain[:target]
    right = domain[target:]
    if len(left) == 0:
        tt = right.index[0]
    elif len(right) == 0:
        tt = left.index[-1]
    else:
        candidate = [right.index[0],left.index[-1]]
        dev = [abs(x-target) for x in candidate]
        if dev[0] <= dev[1]:
            tt = candidate[0]
        else:
            tt = candidate[1]
    if output == True:
        print("@Function [grabNearest] grabbed tt at %s"%tt)
    if isinstance(domain,pd.Series):
        return domain[tt]
    elif isinstance(domain,pd.DataFrame):
        return domain[tt].values
    else:
        raise ValueError

In [2]:
#Configuration
start = datetime(1990,1,1)
end = datetime(2008,1,1)
regressStart = datetime(1990,1,1)
regressEnd = datetime(2008,1,1)
pin_date = 15      #The standing date for month frequecy data entry.

#### Get the output gap time series

In [3]:
#Data inquiry
otpt_raw = pd.read_excel("data/production.xlsx")
otpt = pd.Series(otpt_raw.iloc[:,1].values,index=otpt_raw.iloc[:,0].values,name="industrial_output")[start:end]

In [4]:
#Fitting the quadratic model
regress = pd.DataFrame({'output':otpt})
regress['time'] = np.arange(len(regress))
regress['timesq'] = regress['time']**2
regress['constant'] = 1
model = sm.OLS(regress['output'],regress.drop('output',axis=1))
gapResult = model.fit(cov_type="HC3")
#Get the residual series
output_gap = gapResult.resid
output_gap.index = pd.to_datetime(output_gap.index)
output_gap.name = "outputgap"

In [5]:
#Set the common time line
timeline = output_gap.index

#### Get the inflation time series

In [6]:
#Data inquiry
infl_raw = pd.read_excel("data/inflation.xlsx",skiprows=3)
cpi = pd.Series(infl_raw.iloc[:,1].values,index=infl_raw.iloc[:,0].values,name="cpi")[flashBack(start,1):end]
infl = cpi.pct_change().dropna()

#### Get the federal funds rate

In [7]:
#Data inquiry
iff_raw = pd.read_excel("data/FEDFUNDS.xls",skiprows=10)
iff_tmp = pd.Series(iff_raw['FEDFUNDS'].values,index=iff_raw['observation_date'].values,name = "iff")

In [8]:
#Shaping the series, awkward.
iff1 = pd.Series(np.nan,index=timeline)
for time in timeline:
    iff1[time] = grabForward(target=time,domain=iff_tmp)
head = flashBack(timeline[0],1)
iff1[head] = grabForward(head,iff_tmp)
iff1 = iff1.sort_index()
iff = iff1[iff1.index[1:]]
iff_lag = iff1.shift().dropna()
iff.name = "iffrate"
iff_lag.name = "iffratelag"

#### Run the Taylor's model.

In [9]:
regress = pd.DataFrame([iff,output_gap,infl,iff_lag]).T
regress['Constant'] = 1
taylorModel = sm.OLS(regress.iffrate,regress.drop('iffrate',axis=1))
taylorResult = taylorModel.fit(cov_type="HC3")

#### Peel out the monetary shock of US

In [10]:
monetaryShock = taylorResult.resid
monetaryShock.name = "monetaryShock"

### Part II : Get the UIP deviation estimation

In [11]:
"""The way to estimate the deviation from UIP equation is to use a forward difference of spot exchange rate"""

'The way to estimate the deviation from UIP equation is to use a forward difference of spot exchange rate'

In [12]:
#Data inquiry
exch_raw = pd.read_excel("data/exchange_rate.xlsx")
exch = pd.Series(exch_raw["US $ TO UK £ (WMR) - EXCHANGE RATE"].values,index=exch_raw['Name'].values,name="PoundDollarPrice")

In [13]:
#Shaping the p_t series
pt = pd.Series(np.nan,index=timeline)
for time,value in pt.iteritems():
    print(time)
    pt[time] = grabNearest(target=datetime(time.year+1,time.month,time.day),domain=exch) - grabNearest(time,exch,True)

1990-01-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-01-12 00:00:00
1990-02-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-02-12 00:00:00
1990-03-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-03-12 00:00:00
1990-04-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-04-12 00:00:00
1990-05-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-05-12 00:00:00
1990-06-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-06-12 00:00:00
1990-07-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-07-12 00:00:00
1990-08-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-08-12 00:00:00
1990-09-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-09-12 00:00:00
1990-10-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-10-12 00:00:00
1990-11-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-11-12 00:00:00
1990-12-15 00:00:00
@Function [grabNearest] grabbed tt at 1990-12-12 00:00:00
1991-01-15 00:00:00
@Function [grabNearest] grabbed tt at 1991-0

In [14]:
#Regress pt on monetaryShock
if not (pt.index == monetaryShock.index).all(): #Check the index
    raise ValueError
indep = pd.DataFrame({"monetaryShock":monetaryShock,"Constant":1})
model1 = sm.OLS(pt,indep)
result1 = model1.fit(cov_type="HC3")
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.458
Date:                Tue, 27 Nov 2018   Prob (F-statistic):             0.0643
Time:                        16:45:27   Log-Likelihood:                 106.78
No. Observations:                 216   AIC:                            -209.6
Df Residuals:                     214   BIC:                            -202.8
Df Model:                           1                                         
Covariance Type:                  HC3                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
monetaryShock     0.1064      0.057      1.860      0.063      -0.006       0.219
Constant          0.0036      0.010      0.355      0.723      -0.016       0.023
==============================================================================
Omnibus:                       25.030   Durbin-Watson:                   0.214
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.403
Skew:                          -0.766   Prob(JB):                     9.20e-08
Kurtosis:                       4.119   Cond. No.                         5.43
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [15]:
#Adding a exchange rate lag term
exch_lag = exch - exch.shift()
exch_lag = exch_lag[start:end]
intervn = pd.Series([grabNearest(x,exch_lag) for x in timeline],index=timeline,name="intervention")
indep["intervention"] = intervn
model2 = sm.OLS(pt,indep)
result2 = model2.fit(cov_type="HC3")

In [16]:
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     2.159
Date:                Tue, 27 Nov 2018   Prob (F-statistic):              0.118
Time:                        16:45:28   Log-Likelihood:                 107.24
No. Observations:                 216   AIC:                            -208.5
Df Residuals:                     213   BIC:                            -198.4
Df Model:                           2                                         
Covariance Type:                  HC3                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
monetaryShock     0.1039      0.057      1.811      0.070      -0.009       0.216
Constant          0.0040      0.010      0.397      0.692      -0.016       0.024
intervention     -0.2067      0.224     -0.922      0.356      -0.646       0.232
==============================================================================
Omnibus:                       24.170   Durbin-Watson:                   0.186
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.992
Skew:                          -0.749   Prob(JB):                     1.86e-07
Kurtosis:                       4.095   Cond. No.                         21.5
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""